In [1]:
try:
    import osmnx as ox
except ImportError:
    !pip install --break-system-packages osmnx
    import osmnx as ox
except ModuleNotFoundError:
    !pip install osmnx
    import osmnx as ox 

In [2]:
# 광진구의 골목길 포함 도로 네트워크 가져오기 (residential, service, tertiary 도로 포함)
place_name = "광진구, 서울, 대한민국"
graph = ox.graph_from_place(place_name, network_type='all')


In [3]:

# 모든 도로의 시작점과 종료점 노드 정보
nodes, edges = ox.graph_to_gdfs(graph)


In [4]:
# edges 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(edges.head())
print(edges.info())

# nodes 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(nodes.head())
print(nodes.info())


                                                       osmid      highway  \
u         v           key                                                   
368702043 4104262755  0    [175320385, 175320419, 175320429]     tertiary   
          8963165265  0                            516647748  residential   
          436866074   0                            516647752  residential   
414683232 11076288632 0                           1193259435      primary   
          11125576999 0                           1199529109    secondary   

                                     name  oneway reversed      length tunnel  \
u         v           key                                                       
368702043 4104262755  0    [광장앞지하차도, 광장로]   False    False  430.542185    yes   
          8963165265  0              워커힐로   False    False  107.043667    NaN   
          436866074   0              워커힐로   False     True  177.203342    NaN   
414683232 11076288632 0               자양로    True    Fa

In [17]:
# 전역 배열 초기화
road_names = []
highway_types = []
start_lats = []
start_lons = []
end_lats = []
end_lons = []
tunnel_status = []  # 빈 리스트로 초기화
bridge_status = []  # 빈 리스트로 초기화
service_status = []  # 빈 리스트로 초기화
road_ids = []
max_speeds = []
lane_counts = []
one_way = []
junction_status = []

# 도로 종류 한글 매핑 딕셔너리
road_type_map = {
    'motorway': '고속도로',
    'trunk': '간선도로',
    'primary': '주도로',
    'secondary': '2차로',
    'tertiary': '3차로',
    'unclassified': '비분류 도로',
    'residential': '주거지 도로',
    'service': '서비스 도로',
    'living_street': '생활도로'
}

# 'edges' 데이터프레임에서 각 도로에 대해 정보 추출
for index, row in edges.iterrows():
    # MultiIndex에서 u와 v 값 추출
    start_node = index[0]  # 시작 노드 (u)
    end_node = index[1]    # 종료 노드 (v)

    # 노드 정보를 기반으로 위도와 경도 가져오기
    try:
        start_lat, start_lon = nodes.loc[start_node, ['y', 'x']]
        end_lat, end_lon = nodes.loc[end_node, ['y', 'x']]
    except KeyError:
        # 노드 정보가 없을 경우 건너뛰기
        continue

    # 도로명(속성 name)과 도로 특성 확인
    road_name = row.get('name', None)  # 도로명이 없는 경우 None 처리
    highway_type = row.get('highway', None)  # 도로 종류 (highway)
    tunnel_status_val = row.get('tunnel', None)  # 터널 여부
    bridge_status_val = row.get('bridge', None)  # 교량 여부
    service_status_val = row.get('service', None)  # 서비스 도로 여부
    maxspeed = row.get('maxspeed', None)  # 속도 제한
    lanes = row.get('lanes', None)  # 차선 수
    oneway = row.get('oneway', None)  # 일방통행 여부
    junction = row.get('junction', None)  # 교차로 여부

    # highway_type이 리스트인 경우 첫 번째 값만 사용
    if isinstance(highway_type, list):
        highway_type = highway_type[0]
    
    # 전역 배열에 데이터 추가
    road_names.append(road_name if road_name else None)  # 값이 없으면 None로 처리
    # 도로 종류를 한글로 매핑
    road_type_in_korean = road_type_map.get(highway_type, None)
    highway_types.append(road_type_in_korean if road_type_in_korean else None)  # 값이 없으면 None로 처리
    start_lats.append(start_lat if start_lat else None)  # 값이 없으면 None로 처리
    start_lons.append(start_lon if start_lon else None)  # 값이 없으면 None로 처리
    end_lats.append(end_lat if end_lat else None)  # 값이 없으면 None로 처리
    end_lons.append(end_lon if end_lon else None)  # 값이 없으면 None로 처리
    tunnel_status.append(tunnel_status_val if tunnel_status_val else None)  # 값이 없으면 None로 처리
    bridge_status.append(bridge_status_val if bridge_status_val else None)  # 값이 없으면 None로 처리
    service_status.append(service_status_val if service_status_val else None)  # 값이 없으면 None로 처리
    road_ids.append(row['osmid'] if row['osmid'] else None)  # 값이 없으면 None로 처리
    max_speeds.append(maxspeed if maxspeed else None)  # 값이 없으면 None로 처리
    lane_counts.append(lanes if lanes else None)  # 값이 없으면 None로 처리
    one_way.append(oneway if oneway else None)  # 값이 없으면 None로 처리
    junction_status.append(junction if junction else None)  # 값이 없으면 None로 처리

# 각 배열의 길이 확인
print(len(road_names), len(highway_types), len(start_lats), len(start_lons))

# 데이터프레임 생성
data = {
    '도로명': road_names,
    '도로 종류': highway_types,
    '시작점_위도': start_lats,
    '시작점_경도': start_lons,
    '종료점_위도': end_lats,
    '종료점_경도': end_lons,
    '터널 여부': tunnel_status,
    '교량 여부': bridge_status,
    '서비스 도로 여부': service_status,
    '도로 ID': road_ids,
    '속도 제한': max_speeds,
    '차선 수': lane_counts,
    '일방통행 여부': one_way,
    '교차로 여부': junction_status
}

df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('./make_file/광진구_도로정보_전체.csv', index=False)

print("광진구의 도로 정보가 CSV 파일로 저장되었습니다.")


14461 14461 14461 14461
광진구의 도로 정보가 CSV 파일로 저장되었습니다.


,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부
0,"[광장앞지하차도, 광장로]",3차로,37.545509,127.097772,37.546747,127.102031,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,None,None
1,워커힐로,주거지 도로,37.545509,127.097772,37.546447,127.098044,NaN,NaN,NaN,516647748,NaN,NaN,None,None
2,워커힐로,주거지 도로,37.545509,127.097772,37.543981,127.097208,NaN,NaN,NaN,516647752,NaN,NaN,None,None
3,자양로,주도로,37.536546,127.083544,37.536399,127.083587,NaN,NaN,NaN,1193259435,NaN,3,True,None
4,아차산로,2차로,37.536546,127.083544,37.536555,127.083667,NaN,NaN,NaN,1199529109,50,3,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14456,동일로30길,주거지 도로,37.547623,127.068077,37.547628,127.067994,NaN,NaN,NaN,381409375,NaN,NaN,None,None
14457,NaN,생활도로,37.547623,127.068077,37.547807,127.068116,NaN,NaN,NaN,1369902935,NaN,1,None,None
14458,NaN,생활도로,37.547807,127.068116,37.547623,127.068077,NaN,NaN,NaN,1369902935,NaN,1,None,None
14459,NaN,생활도로,37.547807,127.068116,37.548165,127.068194,NaN,NaN,NaN,1369902935,NaN,1,None,None
